<a href="https://colab.research.google.com/github/tutu-tonton/Python-SQLAlchemy_tut/blob/main/SQLAlchemy_tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [2]:
import pandas as pd
import os

In [3]:
database_file = os.path.join(os.path.abspath(os.getcwd()), 'data.db')

In [ ]:
database_file

In [ ]:
engine = create_engine('sqlite:///' + database_file, convert_unicode=True, echo=True)

In [6]:
db_session = scoped_session(
    sessionmaker(
        autocommit = False,
        autoflush = False,
        bind = engine
    )
)

In [7]:
Base = declarative_base()
Base.query = db_session.query_property()

In [8]:
from sqlalchemy import Column, Integer, Float

In [9]:
class Wine(Base):
  __tablename__ = 'wine_class'
  id = Column(Integer, primary_key=True)
  wine_class = Column(Integer, unique=False)
  alcohol = Column(Float, unique=False)
  ash = Column(Float, unique=False)
  hue = Column(Float, unique=False)
  proline = Column(Integer, unique=False)

  def __init__(self, wine_class=None, alcohol=None, ash=None, hue=None, proline=None):
    self.wine_class = wine_class
    self.alcohol = alcohol
    self.ash = ash
    self.hue = hue
    self.proline = proline

In [10]:
Base.metadata.create_all(bind=engine)

2021-11-07 06:49:35,677 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-07 06:49:35,680 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("wine_class")
2021-11-07 06:49:35,682 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-07 06:49:35,685 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
def read_data():
  wine_df = pd.read_csv('/content/drive/MyDrive/wine_class.csv')

  for index, _df in wine_df.iterrows():
    row = Wine(wine_class=_df['Class'], alcohol=_df['Alcohol'], ash=_df['Ash'], hue=_df['Hue'], proline=_df['Proline'])
    db_session.add(row)
    db_session.commit()

In [ ]:
read_data()

==================== ここまで準備 ====================

## 全データ取得

In [ ]:
db = db_session.query(Wine).all()

In [ ]:
for row in db:
  print(row.alcohol)

 ## 一部カラムのみ取得

In [ ]:
db = db_session.query(Wine.hue, Wine.proline).all()

In [ ]:
for row in db:
  print(row.proline)

## 条件抽出

In [ ]:
db = db_session.query(Wine).filter(Wine.hue > 1.0).all()

In [ ]:
for row in db:
  print(row.hue)

## 取得レコード件数制限

In [ ]:
db = db_session.query(Wine).limit(20).all()

In [ ]:
for row in db:
  print(row.hue)

## レコード並べ替え

In [23]:
from sqlalchemy import desc

In [25]:
db = db_session.query(Wine).order_by(desc(Wine.hue)).limit(5).all()

for row in db:
  print(row.hue)

2021-11-07 07:12:04,029 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class ORDER BY wine_class.hue DESC
 LIMIT ? OFFSET ?
2021-11-07 07:12:04,032 INFO sqlalchemy.engine.Engine [generated in 0.00278s] (5, 0)
1.71
1.71
1.45
1.45
1.42


## 新規データ追加

In [26]:
new_wine = Wine(wine_class=1, alcohol=1, ash=1, hue=1, proline=1)

db_session.add(new_wine)
db_session.commit()

2021-11-07 07:15:22,733 INFO sqlalchemy.engine.Engine INSERT INTO wine_class (wine_class, alcohol, ash, hue, proline) VALUES (?, ?, ?, ?, ?)
2021-11-07 07:15:22,735 INFO sqlalchemy.engine.Engine [generated in 0.00297s] (1, 1.0, 1.0, 1.0, 1)
2021-11-07 07:15:22,740 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
db = db_session.query(Wine).filter(Wine.proline == 1).all()

for row in db:
  print(row.id, row.wine_class, row.alcohol, row.ash, row.hue, row.proline)

2021-11-07 07:19:50,091 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class 
WHERE wine_class.proline = ?
2021-11-07 07:19:50,095 INFO sqlalchemy.engine.Engine [cached since 115.7s ago] (1,)
357 1 1.0 1.0 1.0 1


## データ更新

In [ ]:
db = db_session.query(Wine).filter(Wine.proline == 1).first()

In [31]:
db.wine_class = 10

In [33]:
db_session.commit()

2021-11-07 07:23:12,822 INFO sqlalchemy.engine.Engine UPDATE wine_class SET wine_class=? WHERE wine_class.id = ?
2021-11-07 07:23:12,830 INFO sqlalchemy.engine.Engine [generated in 0.00897s] (10, 357)
2021-11-07 07:23:12,834 INFO sqlalchemy.engine.Engine COMMIT


## データ削除

In [ ]:
db = db_session.query(Wine).filter(Wine.proline == 1).delete()